In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('31151.csv')

In [3]:
df

,timestamp,source,host,response_status,request_time,http_user_agent,message
0,2023-08-02T06:48:57.000Z,178.216.100.193,suggestions.dadata.ru,200,0.001,GuzzleHttp/7,GET /suggestions/api/4_1/rs/iplocate/address?i...
1,2023-08-02T06:48:57.000Z,178.216.100.193,suggestions.dadata.ru,200,0.002,GuzzleHttp/7,GET /suggestions/api/4_1/rs/iplocate/address?i...
2,2023-08-02T06:48:57.000Z,178.216.100.193,suggestions.dadata.ru,200,0.001,GuzzleHttp/7,GET /suggestions/api/4_1/rs/iplocate/address?i...
3,2023-08-02T06:48:57.000Z,178.216.100.193,suggestions.dadata.ru,200,0.006,GuzzleHttp/7,GET /suggestions/api/4_1/rs/iplocate/address?i...
4,2023-08-02T06:48:57.000Z,178.216.100.193,suggestions.dadata.ru,200,0.002,GuzzleHttp/7,GET /suggestions/api/4_1/rs/iplocate/address?i...
...,...,...,...,...,...,...,...
11880,2023-08-02T12:29:41.000Z,178.216.98.64,suggestions.dadata.ru,403,0.001,GuzzleHttp/7,GET /suggestions/api/4_1/rs/iplocate/address?i...
11881,2023-08-02T12:30:26.000Z,89.23.156.102,suggestions.dadata.ru,403,0.008,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,POST /suggestions/api/4_1/rs/suggest/address H...
11882,2023-08-02T12:30:26.000Z,89.23.156.102,suggestions.dadata.ru,403,0.007,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,POST /suggestions/api/4_1/rs/suggest/address H...
11883,2023-08-02T14:05:03.000Z,90.154.71.17,suggestions.dadata.ru,403,0.009,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,POST /suggestions/api/4_1/rs/suggest/address H...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11885 entries, 0 to 11884
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   timestamp        11885 non-null  object 
 1   source           11885 non-null  object 
 2   host             11885 non-null  object 
 3   response_status  11885 non-null  int64  
 4   request_time     11885 non-null  float64
 5   http_user_agent  11885 non-null  object 
 6   message          11885 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 650.1+ KB


In [11]:
df.nunique()

timestamp          2234
source                9
host                  2
response_status       2
request_time         28
http_user_agent       5
message              17
dtype: int64

In [12]:
df.source.unique()

array(['178.216.100.193', '188.35.29.1', '178.216.98.64', '217.73.62.199',
       '195.38.23.9', '90.154.71.17', '109.252.176.52', '83.220.238.14',
       '89.23.156.102'], dtype=object)

In [14]:
df.source.value_counts()

178.216.98.64      5958
217.73.62.199      5832
90.154.71.17         40
178.216.100.193      35
195.38.23.9           8
109.252.176.52        6
188.35.29.1           2
83.220.238.14         2
89.23.156.102         2
Name: source, dtype: int64

https://www.freecodecamp.org/news/how-to-get-location-information-of-ip-address-using-python/